In [1]:
import yaml
import src.blocks as blocks
from src import Block
import src.merge as merge
from collections import OrderedDict
from jinja2 import Environment, FileSystemLoader

In [2]:
# Read configuration
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config


{'gen1': {'a': 2, 'b': 10, 'c': 4}, 'gen2': {'d': 0.5}}

In [3]:
# Read master
with open("master.yaml", "r") as f:
    master = yaml.load(f, Loader=yaml.FullLoader)


In [4]:
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    for block in master[gen]['script']:
        print("Now browsing block: " + block)
        print(master[gen]['script'][block])
    print()

Now browsing generation: gen1
Now browsing block: multiply
{'args': ['b', 'c'], 'output': 'bc'}
Now browsing block: add
{'args': ['a', 'bc'], 'output': 'a_bc'}
Now browsing block: factorial
{'args': 'a_bc', 'output': 'fact_a_bc'}



In [8]:
def generate_main(script, config, dict_blocks):

    # Generate header
    header_str = "def main():\n"

    # Declare parameters
    parameters_str = "\t# Declare parameters\n"
    for param, value in config.items():
        parameters_str += "\t" + param + " = " + str(value) + "\n"

    # Declare blocks
    blocks_str = "\n\t# Declare blocks\n"
    for block, dict_block in script.items():
        # Get block arguments
        l_args = dict_block["args"]
        if not isinstance(l_args, list):
            l_args = [l_args]
        # Get block outputs
        l_outputs = dict_block["output"]
        if not isinstance(l_outputs, list):
            l_outputs = [l_outputs]
        # Write blocks string
        blocks_str += "\t" + dict_blocks[block].get_assignation_call_str(l_args, l_outputs) + "\n"

    # Save output
    # ! TODO

    return header_str + parameters_str + blocks_str

In [9]:
# Generate generations from template
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("default_template.txt")

for gen in sorted(master.keys()):
    file_path = f"{gen}.py"
    print("Now browsing generation: " + gen)
    print('\n')
    dict_blocks = OrderedDict()
    for block in master[gen]['script']:
        dict_blocks[block] = getattr(blocks,block)
            
    # Merge all imports
    dict_imports_merge = merge.merge_imports(list(dict_blocks.values()))
    
    # Get string imports
    str_imports = Block.get_external_l_imports_str(dict_imports_merge)
    
    # Get the dictionnary of block strings
    dict_blocks_str = {k: v.get_str() for k,v in dict_blocks.items()}
    
    # Get corresponding block string
    str_blocks = "\n".join([f"{k}" for k in dict_blocks_str.values()])
    
    # Write main script
    str_main = generate_main(master[gen]['script'], config[gen], dict_blocks)
    
    # Render template
    content = template.render(
        imports = str_imports,
        blocks = str_blocks,
        main=str_main,
    )
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(content)
    
                

Now browsing generation: gen1




In [7]:
# Try merging blocks

# Add takes a and b and returns output_add
# Power takes b and c and returns output_power
# Let's add to output c instead of output add
from typing import OrderedDict

dic_blocks["add"].output = OrderedDict([("c", dic_blocks["add"].output["output_add"])])

add_power = dic_blocks["add"].merge_block(
    dic_blocks["power"],
    "add_power",
    docstring="Test with long docstring",
    output = dic_blocks["power"].output,
)

NameError: name 'dic_blocks' is not defined

In [9]:
print('*********')
print(add_power.get_str())
print('*********')
print(add_power.get_output_str())
print('*********')
print(add_power.get_output_type_hint_str())
print('*********')
print(add_power.get_call_str())
print('*********')
print(add_power.get_assignation_call_str())
print('*********')
print(add_power.get_signature())
print('*********')
print(add_power.parameters)
print('*********')
print(add_power.dict_imports)
print('*********')
print(add_power.set_deps)


*********
def add_power(a: int, b: int) -> int:
    """Test with long docstring
    """

    c = add_function(a, b)
    output_power = power_function(b, c)
    return output_power

*********
output_power
*********
int
*********
add_power(a, b)
*********
output_power = add_power(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('numpy', 'np')])
*********
{'power_function', 'add_function'}


In [10]:
print(dic_blocks["add"].output)

OrderedDict([('c', <class 'int'>)])


In [11]:
# Now with several blocks
from src import Block

dic_blocks["add"].parameters = OrderedDict([("a", dic_blocks["add"].output["c"]), ("output_factorial", dic_blocks["factorial"].output["output_factorial"])])
dic_blocks["multiply"].output = OrderedDict([("c", dic_blocks["multiply"].output["output_multiply"])])
dic_blocks["power"].output = OrderedDict([("d", dic_blocks["power"].output["output_power"])])
dic_blocks["print_result"]

test_multiple = Block.merge_blocks(
    [dic_blocks["factorial"], dic_blocks["add"], dic_blocks["multiply"], dic_blocks["power"], dic_blocks["print_result"]],
    "test_multiple_blocks",
    docstring="This function contains many blocks",
    output = dic_blocks["power"].output,
)

In [12]:
print('*********')
print(test_multiple.get_str())
print('*********')
print(dic_blocks["add"].get_str())
print('*********')
print(test_multiple.get_output_str())
print('*********')
print(test_multiple.get_output_type_hint_str())
print('*********')
print(test_multiple.get_call_str())
print('*********')
print(test_multiple.get_assignation_call_str())
print('*********')
print(test_multiple.get_signature())
print('*********')
print(test_multiple.parameters)
print('*********')
print(test_multiple.dict_imports)
print('*********')
print(test_multiple.set_deps)

*********
def test_multiple_blocks(a: int, b: int) -> int:
    """This function contains many blocks
    """

    output_factorial = factorial_function(a)
    c = add_function(a, output_factorial)
    c = multiply_function(a, b)
    d = power_function(b, c)
    print_result_function(d)
    return d

*********
def add_function(a: int, output_factorial: int) -> int:
    """Dummy docstring
    """
    b = output_factorial
    
    # Add a and b
    return a + b

*********
d
*********
int
*********
test_multiple_blocks(a, b)
*********
d = test_multiple_blocks(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('math', 'math'), ('numpy', 'np')])
*********
{'add_function', 'multiply_function', 'factorial_function', 'power_function', 'print_result_function'}
